# 14. REINFORCE and Tree Search on Connect 4


## 지금까지 해온 것들을 모두 모으자!

여기서는 policy gradient (REINFORCE),
tree search with policy를 혼용해서
`agent_greedy`를 뛰어넘는
Connect 4 policy를 강화학습 시켜봅니다.

방식은 다음과 같습니다.

- Episode에 대해서는 Monte Carlo policy gradient를
사용하여 policy를 갱신.
- Policy는 policy network를 base policy로 하여
tree search with policy를 사용.

학습을 할 때 누구와 싸워가며 학습할지를 생각해볼 수 있는데,
이 부분은 조금 뒤에서 다시 얘기하겠습니다.

## 구현

In [1]:
!rm -rf mock4.py m4
!git clone https://github.com/lumiknit/mock4.py.git
!mv mock4.py m4
!mv m4/mock4.py .
import mock4
from mock4 import Mock4

Cloning into 'mock4.py'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 28 (delta 8), reused 15 (delta 4), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [2]:
import matplotlib.pyplot as plt

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: {}".format(device))

Device: cuda


In [3]:
class Scope: pass

### Agent & Policy Transformer

In [4]:
def softmax(arr, tau=1.0):
  arr = np.array(arr, dtype=np.float64)
  arr /= tau
  m = max(arr)
  z = np.exp(arr - m)
  return z / z.sum()

In [5]:
def agent(pi, epsilon=0):
  # pi must return array of non-negative values
  def c(game):
    w, h = game.w, game.h
    m, p = np.ones(game.w), np.array(pi(game))
    for i in range(game.w):
      if game.board[(i + 1) * h - 1] != 0: m[i], p[i] = 0, 0
    s = p.sum()
    if np.random.uniform() < epsilon or s == 0:
      s = m.sum()
      if s == 0: return None # Cannot do anything
      else: return np.random.choice(w, p=(m / s))
    else: return np.random.choice(w, p=(p / s))
  name = str(pi)
  if hasattr(pi, 'name'): name = pi.name
  c.name = "stochastic({})".format(name)
  return c

In [6]:
def pt_softmax(policy, tau=1.0):
  def p(game):
    p = policy(game)
    return softmax(p, tau=tau)
  p_name = mock4.agent_name(policy)
  p.name = 'pt_softmax({},tau={})'.format(p_name, tau)
  return p

아래 TSwB는 이전 챕터의 함수와 거의 동일하지만,
`max_depth`와 `max_breadth`를 받는 대신에
탐색할 detph별로 최대 너비를 list로 입력받습니다.
예를 들어서 처음에는 7가지 경우 모두, 그 다음에는 3가지,
이후 1자기 경로로 깊이 5만큼 탐색하고 싶다면,
`[7, 3, 1, 1, 1, 1, 1]`을 전달하면 됩니다.

In [7]:
# Tree Search with Bound
def pt_tswb(policy, max_breadths):
  def search(game, depth): # return (winner, action)
    # If game is terminated, just return the winner
    w = game.check_win()
    if w is not None or depth >= len(max_breadths):
      if depth == 0: return w, policy(game)
      else: return w, None
    # Search
    # First, find policy
    p = policy(game)
    m_full = np.ones(len(p))
    m_lose = np.ones(len(p))
    m_win = np.zeros(len(p))
    m_searched = np.zeros(len(p))
    # Remove unavailable actions
    for c in range(game.w):
      if game.board[(c + 1) * game.h - 1] != 0:
        m_full[c] = 0
    # Sort policy
    s = sorted([(-x, i) for i, x in enumerate(p * m_full)])
    for mprob, i in s[: max_breadths[depth]]:
      if -mprob <= 0: break
      m_searched[i] = 1
      game.place(i)
      w, j = search(game, depth + 1)
      game.undo()
      if w == game.player: # If player win, we can win follow this.
        m_win[i] = 1
      elif w == 3 - game.player: # If player lose, we should not follow
        m_lose[i] = 0
    if (m_win * p).sum() > 0: # If there are winning way, use them
      return game.player, m_win * p
    if (m_lose * m_full * m_searched * p).sum() > 0: # If there are no losing way
      return None, m_lose * m_full * m_searched * p
    # Otherwise
    return (3 - game.player), m_lose * m_full * p
    
  def p(game):
    winner, p = search(game, 0)
    if p is None: p = policy(game)
    return p
  p.name = 'pt_tswb({},b={})'.format(
      mock4.agent_name(policy), max_breadths)
  return p

### Neural Network

In [8]:
class Flatten(nn.Module):
  def forward(self, x):
    if len(x.shape) == 3: return x.view(-1)
    else: return x.flatten(1, -1)

Neural network는 이전 policy를 저장할 수 있도록
아래와 같이 만듭니다.

In [9]:
## nn
def new_nn():
  W = 7
  H = 6
  net = nn.Sequential(
      # 01
      nn.Conv2d(3, 64, 3, padding='same'),
      nn.ReLU(),
      # 02
      nn.Conv2d(64, 64, 3, padding='same'),
      nn.MaxPool2d(2),
      nn.ReLU(),
      # 03
      nn.Conv2d(64, 32, 3, padding='same'),
      nn.ReLU(),
      # 04
      nn.Conv2d(32, 8, 3, padding='same'),
      nn.ReLU(),
      # Lin01
      Flatten(),
      nn.Linear(8 * (W // 2) * (H // 2), 20),
      nn.ReLU(),
      # Lin02
      nn.Linear(20, W),
      # Softmax
      nn.Softmax(dim=-1)
  ).to(device)
  return net

def init_nn():
  global policy, policy_backs
  policy = new_nn()
  policy_backs = []
  save_policy()

def save_policy():
  global policy_backs
  n = new_nn()
  n.load_state_dict(policy.state_dict())
  policy_backs = (policy_backs + [n])[-10: ]

In [10]:
def policy_model(net):
  def c(game):
    X = game.tensor().unsqueeze(dim=0).to(device)
    with torch.no_grad():
      p = net(X)
    arr = p.squeeze().to('cpu').numpy()
    return arr
  c.name = 'model({:x})'.format(id(net))
  return c

In [11]:
def loss_fn(policy, v):
  # Original GD (minimization) : theta <- theta - alpha D J(theta)
  # In policy gradient (maximization) : theta <- theta + alpha v D log pi
  # => Negative log likelihood weighted by reward v
  return - (v * torch.log(policy)).mean()

In [12]:
class Stat:
  def __init__(self, n=0, w=0, l=0, d=0): self.n, self.w, self.l, self.d = n, w, l, d
  def dup(self):
    return Stat(self.n, self.w, self.l, self.d)
  def __sub__(self, other):
    return Stat(self.n - other.n, self.w - other.w, self.l - other.l, self.d - other.d)

In [17]:
# REINFORCE
def learn(
    agent1_gen,
    agent2_gen,
    opt,
    n_episode,
    n_epoch,
    gamma,
    batch_size,
    interval_stat
):
  epi = 0
  stat = Stat()
  last_stat = Stat()
  last_stat_epi = 0
  Xs, As, Vs = [], [], []
  for epi in range(n_episode):
    # Run Game
    game = Mock4()
    a1 = agent1_gen(epi)
    a2 = agent2_gen(epi)
    result = game.play(a1, a2, p_msg=False, p_res=False)
    # Make reward
    stat.n += 1
    if result == 0: stat.d += 1
    elif result == 1: stat.w += 1
    else: stat.l += 1
    w = 3 - game.player
    reward = 1 if result != 0 else 0.3
    # Append to Batch
    while len(game.history) > 0:
      a = int(game.undo() / game.h)
      if game.player == w:
        S0_p = game.tensor(player=w)
        Xs.append(S0_p)
        As.append(a)
        Vs.append(reward)
        S0_pf = S0_p.flip((1,))
        Xs.append(S0_pf)
        As.append(game.w - a - 1)
        Vs.append(reward)
      reward *= gamma
    # If batch is full enough, perform gradient ascent
    if len(Xs) >= batch_size:
      # Tensor-fy
      X = torch.stack(Xs).to(device)
      A = torch.tensor(As).unsqueeze(dim=1).to(device)
      V = torch.tensor(Vs, dtype=torch.float).to(device)
      Xs, As, Vs = [], [], []
      # Learn
      loss_list = []
      for e in range(n_epoch):
        opt.zero_grad()
        pi_s = policy(X)
        pi_sa = pi_s.gather(1, A).squeeze(dim=1)
        loss = loss_fn(pi_sa, V)
        loss_list.append(loss.mean().item())
        loss.backward()
        opt.step()
      # Print status
      if epi - last_stat_epi >= interval_stat:
        save_policy()
        print("----------")
        print("Ep #{:<6d} Loss {:13.10f} -> {:13.10f}".format(
          epi, loss_list[0], loss_list[-1]))
        print("  Win Rate {:8.4f}% ({}w + {}d + {}l = {})".format(
            100 * (stat.w + stat.d * 0.5) / stat.n,
            stat.w, stat.d, stat.l, stat.n))
        dstat = stat - last_stat
        print("   WR Diff {:8.4f}% ({}w + {}d + {}l = {})".format(
            100 * (dstat.w + dstat.d * 0.5) / dstat.n,
            dstat.w, dstat.d, dstat.l, dstat.n))
        mock4.test_mock4(20, agent1_gen(epi), mock4.agent_greedy)
        last_stat = stat.dup()
        last_stat_epi = epi

### 학습 1 (v.s. Greedy)

In [20]:
 def run():
  init_nn()

  agent1 = lambda e: agent(policy_model(policy))
  agent2 = lambda e: mock4.agent_greedy
  opt = optim.Adam(policy.parameters(), lr=1e-3, weight_decay=1e-6)

  learn(
      agent1_gen = agent1,
      agent2_gen = agent2,
      opt = opt,
      n_episode = 10000,
      n_epoch = 2,
      gamma = 0.99,
      batch_size = 100,
      interval_stat = 500)
run()

----------
Ep #506    Loss  1.2348499298 ->  1.2363787889
  Win Rate   0.3945% (2w + 0d + 505l = 507)
   WR Diff   0.3945% (2w + 0d + 505l = 507)
** Test
* A1 = stochastic(model(7f9d0e05a350))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #1012   Loss  1.1338516474 ->  1.1246111393
  Win Rate   0.4936% (5w + 0d + 1008l = 1013)
   WR Diff   0.5929% (3w + 0d + 503l = 506)
** Test
* A1 = stochastic(model(7f9d0e05a350))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #1512   Loss  0.7100552917 ->  0.6774426699
  Win Rate   0.8592% (13w + 0d + 1500l = 1513)
   WR Diff   1.6000% (8w + 0d + 492l = 500)
** Test
* A1 = stochastic(model(7f9d0e05a350))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #2015   Loss  0.3047559857 ->  0.2788357139
  Win Rate   3.3482% (67w + 1d + 1948l = 2016)
   WR Diff  10.8350% (54w + 1d + 448l = 503)
** Test
* A1 = stochastic(model(7f9d0e05a

10,000번의 게임을 `agent_greedy`와 진행하며,
5,000번째가 넘어갈 때 쯤부터 승률 50%를 넘어가는 것을
확인할 수 있습니다.

#### 평가

In [21]:
mock4.test_mock4(100, mock4.agent_random, agent(policy_model(policy)))
mock4.test_mock4(100, mock4.agent_greedy, agent(policy_model(policy)))

** Test
* A1 = random
* A2 = stochastic(model(7f9d0e05a350))
Total = 100 games
W1 30 (0.300) / Dr 0 (0.000) / W2 70 (0.700)
** Test
* A1 = greedy
* A2 = stochastic(model(7f9d0e05a350))
Total = 100 games
W1 12 (0.120) / Dr 0 (0.000) / W2 88 (0.880)


랜덤을 상대로 낮지 않은 승률을 보여주며,
`agent_greey` 상대로도 비교적 높은 승률을 보장합니다.

In [31]:
mock4.test_mock4(30, mock4.agent_random, agent(pt_softmax(pt_tswb(policy_model(policy), [3, 3, 2, 2]), tau=1e-3)))
mock4.test_mock4(30, mock4.agent_greedy, agent(pt_softmax(pt_tswb(policy_model(policy), [3, 3, 2, 2]), tau=1e-3)))

** Test
* A1 = random
* A2 = stochastic(pt_softmax(pt_tswb(model(7f9d0e05a350),b=[3, 3, 2, 2]),tau=0.001))
Total = 30 games
W1 3 (0.100) / Dr 0 (0.000) / W2 27 (0.900)
** Test
* A1 = greedy
* A2 = stochastic(pt_softmax(pt_tswb(model(7f9d0e05a350),b=[3, 3, 2, 2]),tau=0.001))
Total = 30 games
W1 0 (0.000) / Dr 0 (0.000) / W2 30 (1.000)


학습한 모델에 tree search를 추가하였을 때
더욱 강하다는 것을 확인할 수 있습니다.

In [44]:
mock4.test_mock4(30,
  agent(pt_softmax(pt_tswb(mock4.policy_greedy_connect, [3]), tau=1e-3)),
  agent(policy_model(policy)))

** Test
* A1 = stochastic(pt_softmax(pt_tswb(<function policy_greedy_connect at 0x7f9dd063bcb0>,b=[3]),tau=0.001))
* A2 = stochastic(model(7f9d0e05a350))
Total = 30 games
W1 30 (1.000) / Dr 0 (0.000) / W2 0 (0.000)


만약 greedy connect에 tree search를 섞게 되면
제대로 대응하지 못하는 것을 확인할 수 있습니다.

In [45]:
mock4.test_mock4(30,
  agent(pt_softmax(pt_tswb(mock4.policy_greedy_connect, [3, 3]), tau=1e-3)),
  agent(pt_softmax(pt_tswb(policy_model(policy), [4, 3, 2, 2]), tau=1e-3)))

** Test
* A1 = stochastic(pt_softmax(pt_tswb(<function policy_greedy_connect at 0x7f9dd063bcb0>,b=[3, 3]),tau=0.001))
* A2 = stochastic(pt_softmax(pt_tswb(model(7f9d0e05a350),b=[5, 3, 2, 2]),tau=0.001))
Total = 30 games
W1 25 (0.833) / Dr 1 (0.033) / W2 4 (0.133)


마찬가지로 model쪽에 tree search를 섞어도 승률이
높게 나오지 않음을 확인할 수 있습니다.

이를 통해 model이 `agent_greedy`에 상당히 과적합이
되었을 것이라고 추측해볼 수 있습니다.

### 학습 2 (v.s. Tree Search with Greedy)

In [47]:
 def run():
  init_nn()

  agent1 = lambda e: agent(policy_model(policy))
  agent2 = lambda e: agent(pt_softmax(pt_tswb(mock4.policy_greedy_connect, [3, 3]), tau=1e-3))
  opt = optim.Adam(policy.parameters(), lr=1e-3, weight_decay=1e-6)

  learn(
      agent1_gen = agent1,
      agent2_gen = agent2,
      opt = opt,
      n_episode = 10000,
      n_epoch = 2,
      gamma = 0.99,
      batch_size = 100,
      interval_stat = 500)
run()

----------
Ep #507    Loss  1.7981696129 ->  1.7996817827
  Win Rate   0.0000% (0w + 0d + 508l = 508)
   WR Diff   0.0000% (0w + 0d + 508l = 508)
** Test
* A1 = stochastic(model(7f9d077427d0))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #1013   Loss  1.6840283871 ->  1.6807372570
  Win Rate   0.1972% (2w + 0d + 1012l = 1014)
   WR Diff   0.3953% (2w + 0d + 504l = 506)
** Test
* A1 = stochastic(model(7f9d077427d0))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #1517   Loss  1.7070105076 ->  1.7013781071
  Win Rate   0.2635% (4w + 0d + 1514l = 1518)
   WR Diff   0.3968% (2w + 0d + 502l = 504)
** Test
* A1 = stochastic(model(7f9d077427d0))
* A2 = greedy
Total = 20 games
W1 1 (0.050) / Dr 0 (0.000) / W2 19 (0.950)
----------
Ep #2023   Loss  1.5274075270 ->  1.5214676857
  Win Rate   0.4447% (8w + 2d + 2014l = 2024)
   WR Diff   0.9881% (4w + 2d + 500l = 506)
** Test
* A1 = stochastic(model(7f9d077427d0

#### 평가

In [50]:
mock4.test_mock4(100, mock4.agent_random, agent(policy_model(policy)))
mock4.test_mock4(100, mock4.agent_greedy, agent(policy_model(policy)))
mock4.test_mock4(30,
  agent(pt_softmax(pt_tswb(mock4.policy_greedy_connect, [3, 3]), tau=1e-3)),
  agent(policy_model(policy)))

** Test
* A1 = random
* A2 = stochastic(model(7f9d077427d0))
Total = 100 games
W1 6 (0.060) / Dr 0 (0.000) / W2 94 (0.940)
** Test
* A1 = greedy
* A2 = stochastic(model(7f9d077427d0))
Total = 100 games
W1 79 (0.790) / Dr 1 (0.010) / W2 20 (0.200)
** Test
* A1 = stochastic(pt_softmax(pt_tswb(<function policy_greedy_connect at 0x7f9dd063bcb0>,b=[3, 3]),tau=0.001))
* A2 = stochastic(model(7f9d077427d0))
Total = 30 games
W1 25 (0.833) / Dr 0 (0.000) / W2 5 (0.167)


Tree search를 곁들인 greedy의 경우에는 일반적인 greedy에
비해 더 강력하다보니 10,000번의 게임으로는
충분히 학습이 되지 않는다는 것을 볼 수 있습니다.
다만, 그래도 어느 정도 local optimum에는 가까워지는 듯
하다는 것을 볼 수 있습니다.
(아마 시간을 더 들이거나 complexity를 올리는 방식으로
충분히 학습시키는 것이 가능할 것입니다.)

Tree search에 대해서는 아주 좋은 성능을 보여주지는 않지만
몇몇 이기는 방법을 익힌 것을 볼 수 있으며,
`agent_greedy`나 `agent_random`을 상대할 때에
아주 나쁜 성능을 보이지는 않는다는 점에서, 과적합이 심하지는
않음을 확인할 수 있습니다.

In [52]:
mock4.test_mock4(30,
  agent(pt_softmax(pt_tswb(mock4.policy_greedy_connect, [3, 3]), tau=1e-3)),
  agent(pt_softmax(pt_tswb(policy_model(policy), [3, 3]), tau=1e-3)))

** Test
* A1 = stochastic(pt_softmax(pt_tswb(<function policy_greedy_connect at 0x7f9dd063bcb0>,b=[3, 3]),tau=0.001))
* A2 = stochastic(pt_softmax(pt_tswb(model(7f9d077427d0),b=[3, 3]),tau=0.001))
Total = 30 games
W1 4 (0.133) / Dr 1 (0.033) / W2 25 (0.833)


추가적으로, 학습한 model에 tree search를 곁들이게
되면 같은 깊이의 tree search를 하는 greedy를
압도하는 것을 확인할 수 있습니다.

### 학습 3 (v.s. Random)

In [53]:
 def run():
  init_nn()

  agent1 = lambda e: agent(policy_model(policy))
  agent2 = lambda e: mock4.agent_random
  opt = optim.Adam(policy.parameters(), lr=1e-3, weight_decay=1e-6)

  learn(
      agent1_gen = agent1,
      agent2_gen = agent2,
      opt = opt,
      n_episode = 10000,
      n_epoch = 2,
      gamma = 0.99,
      batch_size = 100,
      interval_stat = 500)
run()

----------
Ep #500    Loss  1.7342803478 ->  1.7342375517
  Win Rate  49.7006% (248w + 2d + 251l = 501)
   WR Diff  49.7006% (248w + 2d + 251l = 501)
** Test
* A1 = stochastic(model(7f9d075e14d0))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #1004   Loss  1.7154291868 ->  1.7152107954
  Win Rate  50.0498% (500w + 6d + 499l = 1005)
   WR Diff  50.3968% (252w + 4d + 248l = 504)
** Test
* A1 = stochastic(model(7f9d075e14d0))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #1508   Loss  1.7534102201 ->  1.7531008720
  Win Rate  49.8012% (748w + 7d + 754l = 1509)
   WR Diff  49.3056% (248w + 1d + 255l = 504)
** Test
* A1 = stochastic(model(7f9d075e14d0))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #2008   Loss  1.6588441133 ->  1.6575635672
  Win Rate  51.2444% (1025w + 9d + 975l = 2009)
   WR Diff  55.6000% (277w + 2d + 221l = 500)
** Test
* A1 = stochastic(mode

#### 평가

학습 자체는 진행이 됩니다만, 랜덤과의 승률도 유의미하게
오르지는 않는 것을 확인할 수 있습니다.

### 학습 4 (v.s. Self)

현재 자기 자신과 대결할 경우 학습의 방향이 틀려서
잘 안 될 수도 있으므로, backup된 policy들과 대결합니다.

In [57]:
 def run():
  init_nn()

  agent1 = lambda e: agent(policy_model(policy))
  agent2 = lambda e: agent(policy_model(policy_backs[np.random.randint(len(policy_backs))]))
  opt = optim.Adam(policy.parameters(), lr=1e-3, weight_decay=1e-6)

  learn(
      agent1_gen = agent1,
      agent2_gen = agent2,
      opt = opt,
      n_episode = 10000,
      n_epoch = 2,
      gamma = 0.99,
      batch_size = 100,
      interval_stat = 500)
run()

----------
Ep #503    Loss  1.7701172829 ->  1.7690320015
  Win Rate  50.0992% (251w + 3d + 250l = 504)
   WR Diff  50.0992% (251w + 3d + 250l = 504)
** Test
* A1 = stochastic(model(7f9d0762ae50))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #1003   Loss  1.6913933754 ->  1.6903479099
  Win Rate  50.3984% (503w + 6d + 495l = 1004)
   WR Diff  50.7000% (252w + 3d + 245l = 500)
** Test
* A1 = stochastic(model(7f9d0762ae50))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #1504   Loss  1.6926116943 ->  1.6898909807
  Win Rate  51.7276% (775w + 7d + 723l = 1505)
   WR Diff  54.3912% (272w + 1d + 228l = 501)
** Test
* A1 = stochastic(model(7f9d0762ae50))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #2007   Loss  1.6310442686 ->  1.6245292425
  Win Rate  52.6643% (1053w + 9d + 946l = 2008)
   WR Diff  55.4672% (278w + 2d + 223l = 503)
** Test
* A1 = stochastic(mode

In [60]:
mock4.test_mock4(100, mock4.agent_random, agent(policy_model(policy)))
mock4.test_mock4(100, mock4.agent_greedy, agent(policy_model(policy)))

** Test
* A1 = random
* A2 = stochastic(model(7f9d0762ae50))
Total = 100 games
W1 15 (0.150) / Dr 0 (0.000) / W2 85 (0.850)
** Test
* A1 = greedy
* A2 = stochastic(model(7f9d0762ae50))
Total = 100 games
W1 98 (0.980) / Dr 0 (0.000) / W2 2 (0.020)


학습 과정을 살펴보면 model이 계속해서 기존의 model에
비해서는 더 강해지는 것을 볼 수 있으며,
`agent_random`에서 비교적 높은 승률을 거두는 것으로
최소한 임의의 선택을 하는 것은 아니라고 볼 수 있습니다.

다만, 자신을 조금씩 개선을 하지만, 어떤 수가 최선에 가까운지를
빠르게 파악하지는 못하기 때문에 학습이 상당히 더딘 것을 볼
수 있으며, 이 때문에 `agent_greedy`는 거의 이기지
못하는 것을 확인할 수 있습니다.

### 학습 5 (v.s. Greedy, then, Tree Search with Greedy)

이 방식으로 실행하면 Gradient explode가 발생하는지,
발산하게 됩니다.

### 학습 6 (v.s. Mix Greedy and Tree Search with Greedy)

In [75]:
 def run():
  init_nn()

  opt = optim.Adam(policy.parameters(), lr=2e-3, weight_decay=1e-6)

  agent1 = lambda e: agent(policy_model(policy))

  op = [
        mock4.agent_greedy,
        agent(pt_softmax(pt_tswb(mock4.policy_greedy_connect, [3, 3]), tau=1e-3))
  ]
  agent2 = lambda e: op[np.random.randint(2)]

  learn(
      agent1_gen = agent1,
      agent2_gen = agent2,
      opt = opt,
      n_episode = 10000,
      n_epoch = 2,
      gamma = 0.99,
      batch_size = 200,
      interval_stat = 500)

run()

----------
Ep #501    Loss  1.6494978666 ->  1.6428221464
  Win Rate   0.3984% (2w + 0d + 500l = 502)
   WR Diff   0.3984% (2w + 0d + 500l = 502)
** Test
* A1 = stochastic(model(7f9cc4758950))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #1014   Loss  1.6123181581 ->  1.6070866585
  Win Rate   0.6897% (7w + 0d + 1008l = 1015)
   WR Diff   0.9747% (5w + 0d + 508l = 513)
** Test
* A1 = stochastic(model(7f9cc4758950))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #1525   Loss  1.6123936176 ->  1.5972832441
  Win Rate   0.5242% (8w + 0d + 1518l = 1526)
   WR Diff   0.1957% (1w + 0d + 510l = 511)
** Test
* A1 = stochastic(model(7f9cc4758950))
* A2 = greedy
Total = 20 games
W1 0 (0.000) / Dr 0 (0.000) / W2 20 (1.000)
----------
Ep #2036   Loss  1.3911645412 ->  1.3772492409
  Win Rate   0.5400% (11w + 0d + 2026l = 2037)
   WR Diff   0.5871% (3w + 0d + 508l = 511)
** Test
* A1 = stochastic(model(7f9cc475895

In [77]:
mock4.test_mock4(100, mock4.agent_random, agent(policy_model(policy)))
mock4.test_mock4(100, mock4.agent_greedy, agent(policy_model(policy)))
mock4.test_mock4(30,
  agent(pt_softmax(pt_tswb(mock4.policy_greedy_connect, [3, 3]), tau=1e-3)),
  agent(policy_model(policy)))

** Test
* A1 = random
* A2 = stochastic(model(7f9cc4758950))
Total = 100 games
W1 4 (0.040) / Dr 0 (0.000) / W2 96 (0.960)
** Test
* A1 = greedy
* A2 = stochastic(model(7f9cc4758950))
Total = 100 games
W1 69 (0.690) / Dr 0 (0.000) / W2 31 (0.310)
** Test
* A1 = stochastic(pt_softmax(pt_tswb(<function policy_greedy_connect at 0x7f9dd063bcb0>,b=[3, 3]),tau=0.001))
* A2 = stochastic(model(7f9cc4758950))
Total = 30 games
W1 26 (0.867) / Dr 2 (0.067) / W2 2 (0.067)


이 경우에는 학습 1, 2의 절충안적인 결과가 나오는 것을
확인할 수 있다.
빠르게 greedy를 이기도록 수렴하지는 않지만,
반대로 tree search가 있는 greedy에 대해서도 어느 정도의
데이터를 쌓는 것을 볼 수 있다.

## 결론

- 두 agent가 대립하는 경우에 강화학습을 하면
상대방이 어떤 agent냐에 따라 수렴하는 속도나 수렴하는
점이 상당히 달라지는 것 같습니다.
- 위 학습에서는 이기는 사람이 두는 방법대로 두도록
학습을 시키기 때문에, 상대방이 강하면 강할수록 실력이 더욱
강해지는 것을 볼 수 있습니다.
이 때문에 만약에 environment나 게임에 대한 정보가
있다면 이것을 적극적으로 활용해서 최대한 강한 상대와 맞붙여
학습을 시켜야 빠르게 실력을 올릴 수 있다고 생각합니다.
- 과적합과 편향 문제가 상당히 심한 것을 볼 수 있습니다.
위의 학습 1과 6을 비교해보면 greedy에 대한 승률은
학습 1이 압도적으로 높지만, 여러 모델과 대결했을 때는
학습 6이 전반적으로 더 좋은 성능을 보여줍니다.
때문에 agent가 어느 정도 제한되어 있지 않다면,
가능한 다양한 agent와 맞붙이며 학습을 하는 것이 필요해보입니다.
이 때문에 replay memory를 두거나
여러 agent를 저장해두고 리그 식으로 대결시켜 학습하는 것이
아닌가 싶습니다.
- Policy network도 그렇지만,
parameteric model을 학습할 때에는
무작정 데이터를 많이 넣기보다는 정말로 필요한 데이터만
넣어서 학습을 시키는 것이 중요한 것 같습니다.
이전 챕터 12의 코드와 여기의 차이점은 이전에는
보드판을 승자 입장에서 본 것 뿐만 아니라 패자 입장에서도
전달하였고, 모든 수를 전부 입력으로 주었다는 것인데,
여기서는 그런 경우를 모두 쳐내어서 과적합이 발생할 정도로
학습하는 것이 가능해졌습니다.
- REINFORCE 알고리즘에서 objective function
$J$중, 임의의 $s$, $a$를 통해 return $G$를 얻었을 때
이것이 $J$에서 차지하는 비중 $j$는
$$j(s, a) = G \pi(s, a) $$
로 나타나고, 이것의 미분을 적분하여 기댓값으로 나타내기 위해서
$$j(s, a) = G \frac{\pi(s, a)}{\pi(s, a)} \pi(s, a)$$
$$ \nabla j = (G \nabla \log \pi(s, a)) \pi(s, a) $$
라고 정리합니다.
이 때문에 만약 G가 음수인 경우에는
$\lim_{x \to 0^+}\log x$와
같은 꼴로 음의 무한대로 급격하게 발산해버리는 문제가 생깁니다.
따라서 보상은 음수가 되면 안 되는데,
이런 경우에는 흔히들 말하는 페널티를 주는 것이 매우 힘듭니다.
(예를 들어 기존에는 이기면 1, 지면 -1을 보상/페널티로
지급했는데, 여기서는 $[0,1]$로 스케일 했습니다.)
이를 다르게 말하자면 위 학습은 '올바른 행동에 대한 확률을
증가'시키는 것은 하지만, '잘못된 행동에 대한 확률을 감소'
시키는 것은 하지 않습니다.
그런데 위는 NLL과 같은 형태이며, NLL에서 class가 2개인
경우에 아래와 같은 형태의 BCE Loss를 사용합니다.
$$j = -y_i\log(p) - (1 - y_i)\log(1 - p)$$
위에 따르면 만약 label $y_i$가 1이면 앞의 $\log(p)$의
곡선을 따라 $p$가 극대화하는 방향으로 이동하고,
label $y_i$가 0이면 뒤의 $\log(1 - p)$의 곡선을 따라
$p$가 극소화하는 방향으로 이동합니다.
위 함수를 $G$, $\pi$에 대해 스케일해보면
$$j = -\frac{G + 1}{2}\log(\pi) - \frac{1 - G}{2}\log(1 - \pi)$$
와 같이 만들어볼 수 있을 것입니다.
다만, 이 식을 사용해서 원래 objective function
$J$가 최대화가 될지는 잘 모르겠습니다.


이 다음에는 다시 원래 문제였던 오목으로 넘어가서
agent를 학습해봅니다.